In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import pickle

In [2]:
call = pd.read_csv("data/Train_call.txt", sep='\t')
clin = pd.read_csv("data/Train_clinical.txt", sep='\t')
print(call.head())
print(clin.head())

   Chromosome   Start     End  Nclone  Array.129  Array.34  Array.67  \
0           1    2927   43870       3          0         0         0   
1           1   85022  216735       4          0         0         0   
2           1  370546  372295       4          0         0         0   
3           1  471671  786483       5          0         0         0   
4           1  792533  907406      13          0         0         0   

   Array.24  Array.22  Array.36  ...  Array.64  Array.89  Array.30  Array.35  \
0         0         0         0  ...         0         0         1         0   
1         0         0         0  ...         0         0         1         0   
2         0         0         0  ...         0         0         1         0   
3         0         0         0  ...         0         1         1         0   
4         0         0         0  ...         0         1         1         0   

   Array.93  Array.10  Array.123  Array.100  Array.134  Array.130  
0         0       

In [3]:
X=call[clin[clin['Subgroup']!='HER2+']['Sample']]
X=X.T.reset_index(drop=True)
X.shape

(68, 2834)

In [4]:
y = clin[clin['Subgroup']!='HER2+']['Subgroup'].reset_index(drop=True)
y.shape

(68,)

In [5]:
k_outer = 10
kf = StratifiedKFold(n_splits=k_outer, random_state=12345,shuffle=True)

outer_cross_val = []
outer_indices = []
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    outer_cross_val.append(((X_train,y_train),(X_test,y_test)))
    outer_indices.append((train_index, test_index))

In [6]:
k_inner = 10
kf = StratifiedKFold(n_splits=k_inner, random_state=12345,shuffle=True)

full_cross_val = []
full_indices = []
for split,indices in zip(outer_cross_val,outer_indices):
    (data_X,data_y),(X_test,y_test) = split
    inner_cross_val = []
    inner_indices = []
    for train_index, val_index in kf.split(data_X, data_y):
        X_train, X_val = X.iloc[train_index,:], X.iloc[val_index,:]
        y_train, y_val = y[train_index], y[val_index]
        inner_cross_val.append(((X_train,y_train),(X_val,y_val)))
        inner_indices.append((train_index, val_index))
    full_cross_val.append((inner_cross_val,(X_test,y_test)))
    full_indices.append((inner_indices,indices[1]))

In [7]:
folds_file = 'data/folds.pickle'
with open(folds_file, 'wb') as fid:
    pickle.dump(full_indices, fid, pickle.HIGHEST_PROTOCOL)